In [1]:
import tensorflow as tf
import os
import random
from datasets import dataset_utils
import sys

In [2]:
def _get_dataset_filename(dataset_dir, split_name, shard_id):
    output_filename = 'flowers_%s_%05d-of-%05d.tfrecord' % (
                        split_name, shard_id, 5)
    return os.path.join(dataset_dir, output_filename)

In [3]:
dataset_dir = '/home/mao/Documents/datasets/'

In [4]:
for spilt_name in ['train', 'validation']:
    for shard_id in range(5):
        output_filename = _get_dataset_filename(
                    dataset_dir, spilt_name, shard_id)
        print(output_filename)
        if not tf.gfile.Exists(output_filename):
            print('data file does exist')

/home/mao/Documents/datasets/flowers_train_00000-of-00005.tfrecord
data file does exist
/home/mao/Documents/datasets/flowers_train_00001-of-00005.tfrecord
data file does exist
/home/mao/Documents/datasets/flowers_train_00002-of-00005.tfrecord
data file does exist
/home/mao/Documents/datasets/flowers_train_00003-of-00005.tfrecord
data file does exist
/home/mao/Documents/datasets/flowers_train_00004-of-00005.tfrecord
data file does exist
/home/mao/Documents/datasets/flowers_validation_00000-of-00005.tfrecord
data file does exist
/home/mao/Documents/datasets/flowers_validation_00001-of-00005.tfrecord
data file does exist
/home/mao/Documents/datasets/flowers_validation_00002-of-00005.tfrecord
data file does exist
/home/mao/Documents/datasets/flowers_validation_00003-of-00005.tfrecord
data file does exist
/home/mao/Documents/datasets/flowers_validation_00004-of-00005.tfrecord
data file does exist


In [5]:
flower_root = os.path.join(dataset_dir, 'flower_photos')
flower_root

'/home/mao/Documents/datasets/flower_photos'

In [6]:
directories = []
class_names = []

In [7]:
for filename in os.listdir(flower_root):
    print(filename)

roses
sunflowers
tulips
LICENSE.txt
dandelion
daisy


In [8]:
for filename in os.listdir(flower_root):
    path = os.path.join(flower_root, filename)
    if os.path.isdir(path):
        directories.append(path)
        class_names.append(filename)

In [9]:
directories, class_names

(['/home/mao/Documents/datasets/flower_photos/roses',
  '/home/mao/Documents/datasets/flower_photos/sunflowers',
  '/home/mao/Documents/datasets/flower_photos/tulips',
  '/home/mao/Documents/datasets/flower_photos/dandelion',
  '/home/mao/Documents/datasets/flower_photos/daisy'],
 ['roses', 'sunflowers', 'tulips', 'dandelion', 'daisy'])

In [10]:
photo_filenames = []

In [11]:
for directory in directories:
    for filename in os.listdir(directory):
        path = os.path.join(directory, filename)
        photo_filenames.append(path)

In [12]:
photo_filenames

['/home/mao/Documents/datasets/flower_photos/roses/8096324039_4db2555490.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/3231873181_faf2da6382.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/5777669976_a205f61e5b.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/8692051081_dffa8709e7_m.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/2535466393_6556afeb2f_m.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/7420699022_60fa574524_m.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/2973256732_1926295f35.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/4765063233_f64440c20b.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/1666341535_99c6f7509f_n.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/21347496068_f4d3339607.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/17302463621_d82be11f01_n.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/9160289562_ab2718d19b.jpg',
 '/home/mao/Documents/datasets/flower_ph

In [13]:
class_names

['roses', 'sunflowers', 'tulips', 'dandelion', 'daisy']

In [14]:
dict(zip(class_names, range(len(class_names))))

{'daisy': 4, 'dandelion': 3, 'roses': 0, 'sunflowers': 1, 'tulips': 2}

In [15]:
class_names_to_ids = dict(zip(class_names, range(len(class_names))))

In [16]:
class_names_to_ids

{'daisy': 4, 'dandelion': 3, 'roses': 0, 'sunflowers': 1, 'tulips': 2}

In [17]:
photo_filenames[0:3]

['/home/mao/Documents/datasets/flower_photos/roses/8096324039_4db2555490.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/3231873181_faf2da6382.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/5777669976_a205f61e5b.jpg']

In [18]:
random.shuffle(photo_filenames)

In [19]:
photo_filenames[0:3]

['/home/mao/Documents/datasets/flower_photos/tulips/5529939805_1679b014e1_n.jpg',
 '/home/mao/Documents/datasets/flower_photos/roses/8524505682_bda885af3a_n.jpg',
 '/home/mao/Documents/datasets/flower_photos/tulips/6267021825_a8316e0dcc_m.jpg']

In [20]:
NUM_VALIDATION = 350

In [21]:
training_filenames = photo_filenames[NUM_VALIDATION:]
validation_filenames = photo_filenames[:NUM_VALIDATION]

In [22]:
len(training_filenames)

3320

In [23]:
len(validation_filenames)

350

In [24]:
class ImageReader(object):
  """Helper class that provides TensorFlow image coding utilities."""

  def __init__(self):
    # Initializes function that decodes RGB JPEG data.
    self._decode_jpeg_data = tf.placeholder(dtype=tf.string)
    self._decode_jpeg = tf.image.decode_jpeg(self._decode_jpeg_data, channels=3)

  def read_image_dims(self, sess, image_data):
    image = self.decode_jpeg(sess, image_data)
    return image.shape[0], image.shape[1], image.shape[2]

  def decode_jpeg(self, sess, image_data):
    image = sess.run(self._decode_jpeg,
                     feed_dict={self._decode_jpeg_data: image_data})
    print(image.shape, image.dtype)
    assert len(image.shape) == 3
    assert image.shape[2] == 3
    return image

In [25]:
photo_filenames[0]

'/home/mao/Documents/datasets/flower_photos/tulips/5529939805_1679b014e1_n.jpg'

In [26]:
image_data = tf.gfile.FastGFile(photo_filenames[0], 'rb').read()

In [27]:
image_data

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x03\x02\x02\x03\x02\x02\x03\x03\x03\x03\x04\x03\x03\x04\x05\x08\x05\x05\x04\x04\x05\n\x07\x07\x06\x08\x0c\n\x0c\x0c\x0b\n\x0b\x0b\r\x0e\x12\x10\r\x0e\x11\x0e\x0b\x0b\x10\x16\x10\x11\x13\x14\x15\x15\x15\x0c\x0f\x17\x18\x16\x14\x18\x12\x14\x15\x14\xff\xdb\x00C\x01\x03\x04\x04\x05\x04\x05\t\x05\x05\t\x14\r\x0b\r\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\x14\xff\xc0\x00\x11\x08\x00\xf0\x01@\x03\x01\x11\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1d\x00\x00\x02\x03\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x06\x07\x04\x05\x08\x03\x02\x01\t\x00\xff\xc4\x00I\x10\x00\x01\x03\x02\x04\x03\x04\x07\x06\x03\x06\x04\x05\x05\x01\x00\x01\x02\x03\x04\x05\x11\x00\x06\x12!\x071A\x13"Qa\x08\x142q\x81\x91\xa1\x15#BR\xb1\xc1b\xd1\xe1\x16$3Cr\xf0\t\x17\xa2\xf1%4c\

In [28]:
image_reader = ImageReader()

In [29]:
sess = tf.Session()

In [30]:
height, width, depth = image_reader.read_image_dims(sess, image_data)

(240, 320, 3) uint8


In [32]:
type(height), type(width), depth

(int, int, 3)

In [32]:
os.path.dirname(photo_filenames[0])

'/home/mao/Documents/datasets/flower_photos/dandelion'

In [33]:
os.path.basename(os.path.dirname(photo_filenames[0]))

'dandelion'

In [34]:
class_name = os.path.basename(os.path.dirname(photo_filenames[0]))

In [35]:
class_name

'dandelion'

In [36]:
class_id = class_names_to_ids[class_name]

In [37]:
class_id

3

In [38]:
example = dataset_utils.image_to_tfexample(
    image_data, b'jpg', height, width, class_id)

In [39]:
example

features {
  feature {
    key: "image/class/label"
    value {
      int64_list {
        value: 3
      }
    }
  }
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\001\000H\000H\000\000\377\342\014XICC_PROFILE\000\001\001\000\000\014HLino\002\020\000\000mntrRGB XYZ \007\316\000\002\000\t\000\006\0001\000\000acspMSFT\000\000\000\000IEC sRGB\000\000\000\000\000\000\000\000\000\000\000\000\000\000\366\326\000\001\000\000\000\000\323-HP  \000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\021cprt\000\000\001P\000\000\0003desc\000\000\001\204\000\000\000lwtpt\000\000\001\360\000\000\000\024bkpt\000\000\002\004\000\000\000\024rXYZ\000\000\002\030\000\000\000\024gXYZ\000\000\002,\000\000\000\024bXYZ\000\000\002@\000\000\000\024dmnd\000\000\002T\000\000\000pdmdd\000\000\002\304\000

In [40]:
type(example)

tensorflow.core.example.example_pb2.Example

In [50]:
sys.stdout.write('asas\n')
sys.stdout.flush()

asas


In [51]:
sys.stdout

In [1]:
import download_and_convert_data